# Experiments with the joining validator

In this notebook we experiment with different configurations of the joining validator. In the previous experiments we have been using the default configuration of the joining validator, namely a RandomForestClassifier with 100 trees. In this notebook we will experiment with different configurations of the joining validator, other classifiers and also trying to adjust the threshold parameter.

In [ ]:
import pandas as pd

from pandas import DataFrame
from typing import List, Dict

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from disjoint_generative_model import DisjointGenerativeModels
from disjoint_generative_model.utils.joining_validator import JoiningValidator
from disjoint_generative_model.utils.joining_strategies import UsingJoiningValidator

def single_mixed_model_experiment(df_train: DataFrame, gms: Dict[str, List[str]], id) -> DataFrame:
    """ Function to do runs of the mixed model. """

    dgms = DisjointGenerativeModels(df_train, gms, worker_id=id*10)
    dgms.joining_strategy = UsingJoiningValidator()

    dgms.join_multiplier = 8    # to ensure high enough resolution

    df_dgms = dgms.fit_generate()

    return df_dgms[:len(df_train)]